In [1]:
import pandas as pd
import dask.dataframe as dd
from tqdm.notebook import tqdm
import numpy as np
import datetime as dt
import duckdb
import wrds
import cleanco
import Levenshtein
import multiprocessing
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
db = wrds.Connection(wrds_username='lzy2lzz')
duckdb.default_connection.execute("SET GLOBAL pandas_analyze_sample=100000")

Loading library list...
Done


In [3]:
# factset_sc = pd.read_stata('./factset_revere_companies/revere_relationships_2003-2023.dta')
# factset_sc = factset_sc.loc[(factset_sc['rel_type'] == 'CUSTOMER') | (factset_sc['rel_type'] == 'SUPPLIER')]
# factset_sc.to_csv('./supply_chain/factset_sc.csv', index=False)

In [4]:
factset_sc = pd.read_csv('./supply_chain/factset_sc.csv')
factset_sc = factset_sc[['start_', 'end_', 'rel_type', 'SOURCE_name', 'source_company_id', 'SOURCE_cusip', 'SOURCE_isin', 'TARGET_name', 'target_company_id', 'TARGET_cusip', 'TARGET_isin']]
factset_sc.loc[factset_sc['end_'] == '4000-01-01 00:00:00', 'end_'] = dt.date(2023, 12, 31)
factset_sc['end_'] = pd.to_datetime(factset_sc['end_'])
factset_sc['start_'] = pd.to_datetime(factset_sc['start_'])
# factset_sc['start_year'] = factset_sc.start_.dt.year
# factset_sc['start_quarter'] = factset_sc.start_.dt.quarter
# factset_sc['end_year'] = factset_sc.end_.dt.year
# factset_sc['end_quarter'] = factset_sc.end_.dt.quarter

C:\Users\Silas Liew\AppData\Local\Temp\ipykernel_36236\90765530.py:1: DtypeWarning: Columns (6,12,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  factset_sc = pd.read_csv('./supply_chain/factset_sc.csv')


In [5]:
sc = pd.DataFrame()
sc['source_company_id'] = factset_sc['source_company_id'].drop_duplicates()
sc['lnk'] = 1
year = pd.DataFrame(
    {'year': range(2003, 2024)}
)
year['lnk'] = 1
sc = sc.merge(year,
              how='left',
              on='lnk')

quarter = pd.DataFrame(
    {'quarter': [1, 2, 3, 4],
     'lnk': [1, 1, 1, 1]}
)
sc = sc.merge(quarter,
              how='left',
              on='lnk')
del year
del quarter
del sc['lnk']
sc['date'] = pd.np.select([sc['quarter'] == 1,
                           sc['quarter'] == 2,
                           sc['quarter'] == 3,
                           sc['quarter'] == 4],
                          ['3-31', '6-30', '9-30', '12-31'])
sc['date'] = sc.apply(lambda x: str(x['year']) + '-' + x['date'], axis=1)
sc['date'] = pd.to_datetime(sc['date'])

C:\Users\Silas Liew\AppData\Local\Temp\ipykernel_36236\1048653308.py:22: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  sc['date'] = pd.np.select([sc['quarter'] == 1,


In [6]:
sc = duckdb.query("""
SELECT a.*, b.*
FROM sc as a
LEFT JOIN factset_sc as b
ON a.source_company_id = b.source_company_id and a.date >= b.start_ and a.date <= b.end_
""").df()
sc = sc.drop(['source_company_id_2', 'start_', 'end_'], axis=1)
sc = sc.loc[sc['target_company_id'].notna()]
sc['SOURCE_cusip'] = sc['SOURCE_cusip'].str[:8]
sc['TARGET_cusip'] = sc['TARGET_cusip'].str[:8]
sc = sc.loc[((sc['SOURCE_cusip'].notna()) | (sc['SOURCE_name'].notna()) | (sc['SOURCE_isin'].notna())) &
            ((sc['TARGET_cusip'].notna()) | (sc['TARGET_name'].notna()) | (sc['TARGET_isin'].notna())) ]
sc['SOURCE_name'] = sc['SOURCE_name'].apply(lambda x: cleanco.basename(x.lower()))
sc['TARGET_name'] = sc['TARGET_name'].apply(lambda x: cleanco.basename(x.lower()))
supplier = sc.loc[sc['rel_type'] == 'SUPPLIER']
customer = sc.loc[sc['rel_type'] == 'CUSTOMER']

In [7]:
# ccmxpf_link = db.get_table(library='crsp', table='ccmxpf_linktable')
# fscrsp_link = db.get_table(library='wrdsapps', table='fscrsplink')
# fscrsp_link = pd.read_csv('./supply_chain/wrdsapps_link_crsp_factset.csv', encoding='ISO8859-1')

In [8]:
supplier_cusip = supplier.loc[(supplier['SOURCE_cusip'].notna()) & (supplier['TARGET_cusip'].notna())]
supplier = supplier.drop(supplier_cusip.index)
supplier_isin = supplier.loc[(supplier['SOURCE_isin'].notna()) & (supplier['TARGET_isin'].notna())]
supplier = supplier.drop(supplier_isin.index)
supplier_ticker = supplier.loc[(supplier['SOURCE_name'].notna()) & (supplier['TARGET_name'].notna())]
supplier = supplier.drop(supplier_ticker.index)

customer_cusip = customer.loc[(customer['SOURCE_cusip'].notna()) & (customer['TARGET_cusip'].notna())]
customer = customer.drop(customer_cusip.index)
customer_isin = customer.loc[(customer['SOURCE_isin'].notna()) & (customer['TARGET_isin'].notna())]
customer = customer.drop(customer_isin.index)
customer_ticker = customer.loc[(customer['SOURCE_name'].notna()) & (customer['TARGET_name'].notna())]
customer = customer.drop(customer_ticker.index)

In [9]:
cik_compustat = pd.read_csv('./supply_chain/cik_compustat.csv')
company_dir_names = pd.read_csv('./supply_chain/wrds_company_dir_names.csv')

In [10]:
supplier_cusip = duckdb.query("""
SELECT a.*, b.gvkey, b.CUSIPH, b.cik, b.FNDATE, b.LNDATE
FROM supplier_cusip as a
LEFT JOIN cik_compustat as b
ON a.SOURCE_cusip = b.CUSIPH and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
supplier_cusip = supplier_cusip.drop(['CUSIPH', 'FNDATE', 'LNDATE', 'cik'], axis=1)
supplier_cusip = supplier_cusip.rename(columns={'gvkey': 'SOURCE_gvkey'})
supplier_cusip = duckdb.query("""
SELECT a.*, b.gvkey, b.CUSIPH, b.cik, b.FNDATE, b.LNDATE
FROM supplier_cusip as a
LEFT JOIN cik_compustat as b
ON a.TARGET_cusip = b.CUSIPH and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
supplier_cusip = supplier_cusip.drop(['CUSIPH', 'FNDATE', 'LNDATE', 'cik'], axis=1)
supplier_cusip = supplier_cusip.rename(columns={'gvkey': 'TARGET_gvkey'})

In [11]:
supplier_isin = duckdb.query("""
SELECT a.*, b.isin, b.cikcode
FROM supplier_isin as a
LEFT JOIN company_dir_names as b
ON a.SOURCE_isin = b.isin
""").df()
supplier_isin = duckdb.query("""
SELECT a.*, b.gvkey, b.cik, b.FNDATE, b.LNDATE
FROM supplier_isin as a
LEFT JOIN cik_compustat as b
ON a.cikcode = b.cik and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
supplier_isin = supplier_isin.drop(['isin', 'cikcode', 'FNDATE', 'LNDATE', 'cik'], axis=1)
supplier_isin = supplier_isin.rename(columns={'gvkey': 'SOURCE_gvkey'})

supplier_isin = duckdb.query("""
SELECT a.*, b.isin, b.cikcode
FROM supplier_isin as a
LEFT JOIN company_dir_names as b
ON a.TARGET_isin = b.isin
""").df()
supplier_isin = duckdb.query("""
SELECT a.*, b.gvkey, b.cik, b.FNDATE, b.LNDATE
FROM supplier_isin as a
LEFT JOIN cik_compustat as b
ON a.cikcode = b.cik and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
supplier_isin = supplier_isin.drop(['isin', 'cikcode', 'FNDATE', 'LNDATE', 'cik'], axis=1)
supplier_isin = supplier_isin.rename(columns={'gvkey': 'TARGET_gvkey'})

In [12]:
customer_cusip = duckdb.query("""
SELECT a.*, b.gvkey, b.CUSIPH, b.cik, b.FNDATE, b.LNDATE
FROM customer_cusip as a
LEFT JOIN cik_compustat as b
ON a.SOURCE_cusip = b.CUSIPH and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
customer_cusip = customer_cusip.drop(['CUSIPH', 'FNDATE', 'LNDATE', 'cik'], axis=1)
customer_cusip = customer_cusip.rename(columns={'gvkey': 'SOURCE_gvkey'})
customer_cusip = duckdb.query("""
SELECT a.*, b.gvkey, b.CUSIPH, b.cik, b.FNDATE, b.LNDATE
FROM customer_cusip as a
LEFT JOIN cik_compustat as b
ON a.TARGET_cusip = b.CUSIPH and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
customer_cusip = customer_cusip.drop(['CUSIPH', 'FNDATE', 'LNDATE', 'cik'], axis=1)
customer_cusip = customer_cusip.rename(columns={'gvkey': 'TARGET_gvkey'})

In [13]:
customer_isin = duckdb.query("""
SELECT a.*, b.isin, b.cikcode
FROM customer_isin as a
LEFT JOIN company_dir_names as b
ON a.SOURCE_isin = b.isin
""").df()
customer_isin = duckdb.query("""
SELECT a.*, b.gvkey, b.cik, b.FNDATE, b.LNDATE
FROM customer_isin as a
LEFT JOIN cik_compustat as b
ON a.cikcode = b.cik and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
customer_isin = customer_isin.drop(['isin', 'cikcode', 'FNDATE', 'LNDATE', 'cik'], axis=1)
customer_isin = customer_isin.rename(columns={'gvkey': 'SOURCE_gvkey'})

customer_isin = duckdb.query("""
SELECT a.*, b.isin, b.cikcode
FROM customer_isin as a
LEFT JOIN company_dir_names as b
ON a.TARGET_isin = b.isin
""").df()
customer_isin = duckdb.query("""
SELECT a.*, b.gvkey, b.cik, b.FNDATE, b.LNDATE
FROM customer_isin as a
LEFT JOIN cik_compustat as b
ON a.cikcode = b.cik and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
customer_isin = customer_isin.drop(['isin', 'cikcode', 'FNDATE', 'LNDATE', 'cik'], axis=1)
customer_isin = customer_isin.rename(columns={'gvkey': 'TARGET_gvkey'})

In [14]:
cik_compustat['coname'] = cik_compustat['coname'].astype(str).apply(lambda x: cleanco.basename(x.lower()))

In [5]:
pol_covid = pd.read_stata('./risk_exposure/08_all_scores/pol_covid.dta')
cc = pd.read_csv('./risk_exposure/08_all_scores/cc_scores.csv')
linktable_name = pd.read_csv('./linktables/cik_compustat_name.csv')

In [34]:
linktable_name = cik_compustat.loc[cik_compustat['gvkey'].isin(pd.concat([pol_covid['gvkey'], cc['gvkey']]).drop_duplicates().dropna())].drop_duplicates()


def fuzzy_match(name, names):
    names = pd.DataFrame(names)
    names['name_start'] = str(name)[:1]
    names['name_start'].astype(str)
    names['start'] = names['name'].apply(lambda x: str(x)[:1])
    names['start'] = names['start'].astype(str)
    names_same_startswith = names.loc[names['start'] == names['name_start'], 'name']
    return process.extractBests(name, names_same_startswith, scorer=fuzz.token_set_ratio, score_cutoff=90)


names = pd.concat([supplier_ticker['SOURCE_name'], supplier_ticker['TARGET_name'], customer_ticker['SOURCE_name'], customer_ticker['TARGET_name']]).drop_duplicates().dropna()
names = pd.DataFrame(names)
names = names.rename(columns={0: 'name'})
names['name'] = names['name'].astype(str)
name = linktable_name['coname'].astype(str)
tqdm.pandas()
linktable_name['coname'] = linktable_name['coname'].progress_apply(lambda x: fuzzy_match(x, names))


def extract_best_match(name):
    try:
        return name[0][0]
    except IndexError:
        return np.nan


linktable_name['coname'] = linktable_name['coname'].progress_apply(lambda x: extract_best_match(x))

  0%|          | 0/15139 [00:00<?, ?it/s]

In [17]:
supplier_ticker = supplier_ticker.drop_duplicates()
customer_ticker = customer_ticker.drop_duplicates()

In [19]:
linktable_name = linktable_name[['coname', 'gvkey', 'LNDATE', 'FNDATE']].drop_duplicates().dropna()

In [22]:
supplier_ticker = duckdb.query("""
SELECT a.*, b.coname, b.gvkey, b.LNDATE, b.FNDATE
FROM supplier_ticker as a
LEFT JOIN linktable_name as b
ON a.SOURCE_name = b.coname and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
supplier_ticker = supplier_ticker.drop(['coname', 'FNDATE', 'LNDATE'], axis=1)
supplier_ticker = supplier_ticker.rename(columns={'gvkey': 'SOURCE_gvkey'})
supplier_ticker = duckdb.query("""
SELECT a.*, b.coname, b.gvkey, b.LNDATE, b.FNDATE
FROM supplier_ticker as a
LEFT JOIN linktable_name as b
ON a.TARGET_name = b.coname and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
supplier_ticker = supplier_ticker.drop(['coname', 'FNDATE', 'LNDATE'], axis=1)
supplier_ticker = supplier_ticker.rename(columns={'gvkey': 'TARGET_gvkey'})

customer_ticker = duckdb.query("""
SELECT a.*, b.coname, b.gvkey, b.LNDATE, b.FNDATE
FROM customer_ticker as a
LEFT JOIN linktable_name as b
ON a.SOURCE_name = b.coname and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
customer_ticker = customer_ticker.drop(['coname', 'FNDATE', 'LNDATE'], axis=1)
customer_ticker = customer_ticker.rename(columns={'gvkey': 'SOURCE_gvkey'})

customer_ticker = duckdb.query("""
SELECT a.*, b.coname, b.gvkey, b.LNDATE, b.FNDATE
FROM customer_ticker as a
LEFT JOIN linktable_name as b
ON a.TARGET_name = b.coname and a.date <= b.LNDATE and a.date >= b.FNDATE
""").df()
customer_ticker = customer_ticker.drop(['coname', 'FNDATE', 'LNDATE'], axis=1)
customer_ticker = customer_ticker.rename(columns={'gvkey': 'TARGET_gvkey'})

In [29]:
supplier_cusip = supplier_cusip.loc[(supplier_cusip['SOURCE_gvkey'].notna()) & (supplier_cusip['TARGET_gvkey'].notna())]
customer_cusip = customer_cusip.loc[(customer_cusip['SOURCE_gvkey'].notna()) & (customer_cusip['TARGET_gvkey'].notna())]
supplier_isin = supplier_isin.loc[(supplier_isin['SOURCE_gvkey'].notna()) & (supplier_isin['TARGET_gvkey'].notna())]
customer_isin = customer_isin.loc[(customer_cusip['SOURCE_gvkey'].notna()) & (customer_isin['TARGET_gvkey'].notna())]
supplier_ticker = supplier_ticker.loc[(supplier_ticker['SOURCE_gvkey'].notna()) & (supplier_ticker['TARGET_gvkey'].notna())]
customer_ticker = customer_ticker.loc[(customer_ticker['SOURCE_gvkey'].notna()) & (customer_ticker['TARGET_gvkey'].notna())]
supplier = pd.concat([supplier_cusip, supplier_isin, supplier_ticker])
customer = pd.concat([customer_cusip, customer_isin, customer_ticker])
supplier = supplier.drop_duplicates()
customer = customer.drop_duplicates()
supplier.to_csv('./factset_revere_companies/fs_supplier.csv', index=False)
customer.to_csv('./factset_revere_companies/fs_customer.csv', index=False)

In [3]:
supplier = supplier[['SOURCE_gvkey', 'TARGET_gvkey', 'year', 'quarter']]
supplier.columns = ['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']
customer = customer[['SOURCE_gvkey', 'TARGET_gvkey', 'year', 'quarter']]
customer.columns = ['firm_gvkey', 'customer_gvkey', 'year', 'quarter']

In [6]:
pol_covid['date'] = pd.to_datetime(pol_covid['date'])
pol_covid['year'] = pol_covid.date.dt.year
pol_covid['month'] = pol_covid.date.dt.month
pol_covid['quarter'] = pd.np.select([(pol_covid['month'] <= 3) & (pol_covid['month'] >= 1),
                                     (pol_covid['month'] <= 6) & (pol_covid['month'] >= 4),
                                     (pol_covid['month'] <= 9) & (pol_covid['month'] >= 7),
                                     (pol_covid['month'] <= 12) & (pol_covid['month'] >= 10)],
                                    [1, 2, 3, 4])
pol_covid = pol_covid[['gvkey', 'year', 'quarter', 'Risk', 'Covid_Exposure']]
cc = cc[['gvkey', 'year', 'quarter', 'cc_expo_ew']]

C:\Users\Silas Liew\AppData\Local\Temp\ipykernel_11132\3741961639.py:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  pol_covid['quarter'] = pd.np.select([(pol_covid['month'] <= 3) & (pol_covid['month'] >= 1),


In [7]:
pol_agg = pd.DataFrame()
covid_agg = pd.DataFrame()
cc_agg = pd.DataFrame()
pol_agg['pol'] = pol_covid.groupby(['gvkey', 'year', 'quarter'])['Risk'].mean()
covid_agg['covid'] = pol_covid.groupby(['gvkey', 'year', 'quarter'])['Covid_Exposure'].mean()
cc_agg['cc'] = cc.groupby(['gvkey', 'year', 'quarter'])['cc_expo_ew'].mean()
pol_agg = pol_agg.reset_index()
covid_agg = covid_agg.reset_index()
cc_agg = cc_agg.reset_index()
pol_agg['gvkey'] = pol_agg['gvkey'].astype(float)
covid_agg['gvkey'] = covid_agg['gvkey'].astype(float)
cc_agg['gvkey'] = cc_agg['gvkey'].astype(float)

In [8]:
supplier = supplier.merge(pol_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(covid_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(cc_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.rename(columns={'pol': 'firm_pol',
                                    'covid': 'firm_covid',
                                    'cc': 'firm_cc'})


supplier = supplier.merge(pol_agg,
                          how='left',
                          left_on=['supplier_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(covid_agg,
                          how='left',
                          left_on=['supplier_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.merge(cc_agg,
                          how='left',
                          left_on=['supplier_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
supplier = supplier.drop(['gvkey'], axis=1)
supplier = supplier.rename(columns={'pol': 'supplier_pol',
                                    'covid': 'supplier_covid',
                                    'cc': 'supplier_cc'})


supplier = supplier.drop_duplicates()
supplier = supplier[supplier['year'] >= 2006]

In [9]:
customer = customer.merge(pol_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(covid_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(cc_agg,
                          how='left',
                          left_on=['firm_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.rename(columns={'pol': 'firm_pol',
                                    'covid': 'firm_covid',
                                    'cc': 'firm_cc'})


customer = customer.merge(pol_agg,
                          how='left',
                          left_on=['customer_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(covid_agg,
                          how='left',
                          left_on=['customer_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.merge(cc_agg,
                          how='left',
                          left_on=['customer_gvkey', 'year', 'quarter'],
                          right_on=['gvkey', 'year', 'quarter'])
customer = customer.drop(['gvkey'], axis=1)
customer = customer.rename(columns={'pol': 'customer_pol',
                                    'covid': 'customer_covid',
                                    'cc': 'customer_cc'})


customer = customer.drop_duplicates()
customer = customer[customer['year'] >= 2006]

In [10]:
supplier = supplier.dropna(subset=['firm_pol', 'firm_covid', 'firm_cc'])
customer = customer.dropna(subset=['firm_pol', 'firm_covid', 'firm_cc'])

In [11]:
supplier['larger_supplier_pol'] = np.where(supplier['supplier_pol'] > supplier['firm_pol'], 1, 0)
supplier['larger_supplier_covid'] = np.where(supplier['supplier_covid'] > supplier['firm_covid'], 1, 0)
supplier['larger_supplier_cc'] = np.where(supplier['supplier_cc'] > supplier['firm_cc'], 1, 0)
customer['larger_customer_pol'] = np.where(customer['customer_pol'] > customer['firm_pol'], 1, 0)
customer['larger_customer_covid'] = np.where(customer['customer_covid'] > customer['firm_covid'], 1, 0)
customer['larger_customer_cc'] = np.where(customer['customer_cc'] > customer['firm_cc'], 1, 0)

In [12]:
supplier.to_csv('./factset_revere_companies/fs_supplier.csv', index=False)
customer.to_csv('./factset_revere_companies/fs_customer.csv', index=False)

In [38]:
larger_supplier_pol_prop = pd.DataFrame()
larger_supplier_pol_prop['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_pol'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_pol_prop['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_supplier_covid_prop = pd.DataFrame()
larger_supplier_covid_prop['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_covid'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_covid_prop['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_supplier_cc_prop = pd.DataFrame()
larger_supplier_cc_prop['prop'] = supplier.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_supplier_cc'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_supplier_cc_prop['firm_count'] = supplier.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_pol_prop = pd.DataFrame()
larger_customer_pol_prop['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_pol'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_pol_prop['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_covid_prop = pd.DataFrame()
larger_customer_covid_prop['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_covid'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_covid_prop['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()
larger_customer_cc_prop = pd.DataFrame()
larger_customer_cc_prop['prop'] = customer.groupby(['year', 'quarter']).apply(lambda x: x.loc[x['larger_customer_cc'] == 1, 'firm_gvkey'].nunique() / x['firm_gvkey'].nunique())
larger_customer_cc_prop['firm_count'] = customer.groupby(['year', 'quarter'])['firm_gvkey'].nunique()

In [39]:
wr = pd.ExcelWriter('./transcripts/scores/props_fs.xlsx')
larger_customer_pol_prop.to_excel(wr, sheet_name='larger_customer_pol_prop_fs')
larger_customer_covid_prop.to_excel(wr, sheet_name='larger_customer_covid_prop_fs')
larger_customer_cc_prop.to_excel(wr, sheet_name='larger_customer_cc_prop_fs')
larger_supplier_pol_prop.to_excel(wr, sheet_name='larger_supplier_pol_prop_fs')
larger_supplier_covid_prop.to_excel(wr, sheet_name='larger_supplier_covid_prop_fs')
larger_supplier_cc_prop.to_excel(wr, sheet_name='larger_supplier_cc_prop_fs')
wr.save()

C:\Users\Silas Liew\AppData\Local\Temp\ipykernel_36236\811183472.py:8: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  wr.save()


In [2]:
supplier = pd.read_csv('./factset_revere_companies/fs_supplier.csv')
customer = pd.read_csv('./factset_revere_companies/fs_customer.csv')

In [3]:
supplier = supplier.dropna()
equal_weighted_first_degree_supplier_score = pd.DataFrame()
equal_weighted_first_degree_supplier_score['ewfdsc_pol'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_pol'].mean()
equal_weighted_first_degree_supplier_score['ewfdsc_covid'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_covid'].mean()
equal_weighted_first_degree_supplier_score['ewfdsc_cc'] = supplier.groupby(['firm_gvkey', 'year', 'quarter'])['supplier_cc'].mean()
equal_weighted_first_degree_supplier_score = equal_weighted_first_degree_supplier_score.reset_index()
customer = customer.dropna()
equal_weighted_first_degree_customer_score = pd.DataFrame()
equal_weighted_first_degree_customer_score['ewfdcc_pol'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_pol'].mean()
equal_weighted_first_degree_customer_score['ewfdcc_covid'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_covid'].mean()
equal_weighted_first_degree_customer_score['ewfdcc_cc'] = customer.groupby(['firm_gvkey', 'year', 'quarter'])['customer_cc'].mean()
equal_weighted_first_degree_customer_score = equal_weighted_first_degree_customer_score.reset_index()

In [23]:
equal_weighted_second_degree_supplier_score = supplier[['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']]
equal_weighted_second_degree_supplier_score = equal_weighted_second_degree_supplier_score.merge(equal_weighted_first_degree_supplier_score,
                                                                                                left_on=['supplier_gvkey', 'year', 'quarter'],
                                                                                                right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                                how='left')
equal_weighted_second_degree_supplier_score.drop(['firm_gvkey_y', 'supplier_gvkey'], axis=1, inplace=True)
equal_weighted_second_degree_supplier_score.dropna(inplace=True)
equal_weighted_second_degree_supplier_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                    'ewfdsc_pol': 'ewsdsc_pol',
                                                    'ewfdsc_covid': 'ewsdsc_covid',
                                                    'ewfdsc_cc': 'ewsdsc_cc'},
                                                   axis='columns',
                                                   inplace=True)
equal_weighted_second_degree_supplier_score = equal_weighted_second_degree_supplier_score.groupby(['firm_gvkey', 'year', 'quarter']).mean().reset_index()
equal_weighted_second_degree_customer_score = customer[['firm_gvkey', 'customer_gvkey', 'year', 'quarter']]
equal_weighted_second_degree_customer_score = equal_weighted_second_degree_customer_score.merge(equal_weighted_first_degree_customer_score,
                                                                                                left_on=['customer_gvkey', 'year', 'quarter'],
                                                                                                right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                                how='left')
equal_weighted_second_degree_customer_score.drop(['firm_gvkey_y', 'customer_gvkey'], axis=1, inplace=True)
equal_weighted_second_degree_customer_score.dropna(inplace=True)
equal_weighted_second_degree_customer_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                    'ewfdcc_pol': 'ewsdcc_pol',
                                                    'ewfdcc_covid': 'ewsdcc_covid',
                                                    'ewfdcc_cc': 'ewsdcc_cc'},
                                                   axis='columns',
                                                   inplace=True)
equal_weighted_second_degree_customer_score = equal_weighted_second_degree_customer_score.groupby(['firm_gvkey', 'year', 'quarter']).mean().reset_index()

In [22]:
equal_weighted_third_degree_supplier_score = supplier[['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']]
equal_weighted_third_degree_supplier_score = equal_weighted_third_degree_supplier_score.merge(equal_weighted_third_degree_supplier_score,
                                                                                              left_on=['supplier_gvkey', 'year', 'quarter'],
                                                                                              right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                              how='left')
equal_weighted_third_degree_supplier_score.drop('firm_gvkey_y', axis=1, inplace=True)
equal_weighted_third_degree_supplier_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                   'supplier_gvkey_x': 'supplier_gvkey',
                                                   'supplier_gvkey_y': 'second_supplier_gvkey'},
                                                  axis='columns',
                                                  inplace=True)
equal_weighted_third_degree_supplier_score = equal_weighted_third_degree_supplier_score.merge(equal_weighted_first_degree_supplier_score,
                                                                                              left_on=['second_supplier_gvkey', 'year', 'quarter'],
                                                                                              right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                              how='left')
equal_weighted_third_degree_supplier_score.drop(['firm_gvkey_y', 'supplier_gvkey', 'second_supplier_gvkey'], axis=1, inplace=True)
equal_weighted_third_degree_supplier_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                   'ewfdsc_pol': 'ewtdsc_pol',
                                                   'ewfdsc_covid': 'ewtdsc_covid',
                                                   'ewfdsc_cc': 'ewtdsc_cc'},
                                                  axis='columns',
                                                  inplace=True)
equal_weighted_third_degree_supplier_score = equal_weighted_third_degree_supplier_score.groupby(['firm_gvkey', 'year', 'quarter']).mean().reset_index()

In [24]:
equal_weighted_third_degree_customer_score = customer[['firm_gvkey', 'customer_gvkey', 'year', 'quarter']]
equal_weighted_third_degree_customer_score = equal_weighted_third_degree_customer_score.merge(equal_weighted_third_degree_customer_score,
                                                                                              left_on=['customer_gvkey', 'year', 'quarter'],
                                                                                              right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                              how='left')
equal_weighted_third_degree_customer_score.drop('firm_gvkey_y', axis=1, inplace=True)
equal_weighted_third_degree_customer_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                   'customer_gvkey_x': 'customer_gvkey',
                                                   'customer_gvkey_y': 'second_customer_gvkey'},
                                                  axis='columns',
                                                  inplace=True)
equal_weighted_third_degree_customer_score = equal_weighted_third_degree_customer_score.merge(equal_weighted_first_degree_customer_score,
                                                                                              left_on=['second_customer_gvkey', 'year', 'quarter'],
                                                                                              right_on=['firm_gvkey', 'year', 'quarter'],
                                                                                              how='left')
equal_weighted_third_degree_customer_score.drop(['firm_gvkey_y', 'customer_gvkey', 'second_customer_gvkey'], axis=1, inplace=True)
equal_weighted_third_degree_customer_score.rename({'firm_gvkey_x': 'firm_gvkey',
                                                   'ewfdcc_pol': 'ewtdcc_pol',
                                                   'ewfdcc_covid': 'ewtdcc_covid',
                                                   'ewfdcc_cc': 'ewtdcc_cc'},
                                                  axis='columns',
                                                  inplace=True)
equal_weighted_third_degree_customer_score = equal_weighted_third_degree_customer_score.groupby(['firm_gvkey', 'year', 'quarter']).mean().reset_index()

In [25]:
firm = pd.concat([supplier[['firm_gvkey', 'year', 'quarter', 'firm_pol', 'firm_covid', 'firm_cc']], customer[['firm_gvkey', 'year', 'quarter', 'firm_pol', 'firm_covid', 'firm_cc']]]).drop_duplicates()
firm = firm.merge(equal_weighted_first_degree_supplier_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_first_degree_customer_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_second_degree_supplier_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_second_degree_customer_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_third_degree_supplier_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()
firm = firm.merge(equal_weighted_third_degree_customer_score,
                  on=['firm_gvkey', 'year', 'quarter'],
                  how='left')
firm = firm.drop_duplicates()

In [112]:
firm.to_csv('./factset_revere_companies/fs_firm.csv')

In [39]:
firm.loc[(firm['ewfdsc_pol'].notna() | firm['ewfdcc_pol'].notna()) & (firm['ewsdcc_pol'].notna() | firm['ewsdsc_pol'].notna()) & (firm['ewtdcc_pol'].notna() | firm['ewtdsc_pol'].notna())]

,firm_gvkey,year,quarter,firm_pol,firm_covid,firm_cc,ewfdsc_pol,ewfdsc_covid,ewfdsc_cc,ewfdcc_pol,...,ewsdsc_cc,ewsdcc_pol,ewsdcc_covid,ewsdcc_cc,ewtdsc_pol,ewtdsc_covid,ewtdsc_cc,ewtdcc_pol,ewtdcc_covid,ewtdcc_cc
0,145046.0,2013,1,96.212718,0.0,0.001013,131.154656,0.0,0.000347,NaN,...,0.000235,NaN,NaN,NaN,51.091684,0.0,0.001002,NaN,NaN,NaN
1,145046.0,2013,2,257.870420,0.0,0.000296,58.801941,0.0,0.000139,NaN,...,0.000189,NaN,NaN,NaN,51.690993,0.0,0.000960,NaN,NaN,NaN
2,145046.0,2013,3,93.840642,0.0,0.000000,61.305987,0.0,0.000148,NaN,...,0.000361,NaN,NaN,NaN,40.050634,0.0,0.000961,NaN,NaN,NaN
3,145046.0,2013,4,186.852387,0.0,0.000253,121.115069,0.0,0.000395,NaN,...,0.000259,NaN,NaN,NaN,46.665124,0.0,0.001343,NaN,NaN,NaN
4,145046.0,2014,1,253.004428,0.0,0.000305,107.707777,0.0,0.000365,NaN,...,0.000000,NaN,NaN,NaN,80.122200,0.0,0.001203,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105174,150937.0,2017,3,39.231071,0.0,0.000000,NaN,NaN,NaN,30.732685,...,NaN,42.435508,0.0,0.000434,NaN,NaN,NaN,64.757376,0.0,0.002453
105175,150937.0,2017,4,0.000000,0.0,0.000000,NaN,NaN,NaN,28.063186,...,NaN,10.782834,0.0,0.000000,NaN,NaN,NaN,46.493160,0.0,0.000570
105176,150937.0,2018,1,19.477990,0.0,0.000000,NaN,NaN,NaN,10.911075,...,NaN,10.292301,0.0,0.000000,NaN,NaN,NaN,44.130043,0.0,0.000790
105177,150937.0,2018,2,68.760028,0.0,0.000000,NaN,NaN,NaN,29.783578,...,NaN,73.015542,0.0,0.000317,NaN,NaN,NaN,63.610574,0.0,0.000933


In [51]:
sc_link = pd.concat([supplier[['firm_gvkey', 'year', 'quarter']], customer[['firm_gvkey', 'year', 'quarter']]]).drop_duplicates()
sc_link = sc_link.merge(supplier[['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']],
                        on=['firm_gvkey', 'year', 'quarter'],
                        how='left')
sc_link.rename({'supplier_gvkey': '1_degree_supplier'}, axis=1, inplace=True)
sc_link.drop_duplicates(inplace=True)
sc_link = sc_link.merge(supplier[['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']],
                        left_on=['1_degree_supplier', 'year', 'quarter'],
                        right_on=['firm_gvkey', 'year', 'quarter'],
                        how='left')
sc_link.drop('firm_gvkey_y', axis=1, inplace=True)
sc_link.rename({'firm_gvkey_x': 'firm_gvkey', 'supplier_gvkey': '2_degree_supplier'}, axis=1, inplace=True)
sc_link.drop_duplicates(inplace=True)
sc_link = sc_link.merge(supplier[['firm_gvkey', 'supplier_gvkey', 'year', 'quarter']],
                        left_on=['2_degree_supplier', 'year', 'quarter'],
                        right_on=['firm_gvkey', 'year', 'quarter'],
                        how='left')
sc_link.drop('firm_gvkey_y', axis=1, inplace=True)
sc_link.rename({'firm_gvkey_x': 'firm_gvkey', 'supplier_gvkey': '3_degree_supplier'}, axis=1, inplace=True)
sc_link.drop_duplicates(inplace=True)
sc_link = sc_link.merge(customer[['firm_gvkey', 'customer_gvkey', 'year', 'quarter']],
                        on=['firm_gvkey', 'year', 'quarter'],
                        how='left')
sc_link.rename({'customer_gvkey': '1_degree_customer'}, axis=1, inplace=True)
sc_link.drop_duplicates(inplace=True)
sc_link = sc_link.merge(customer[['firm_gvkey', 'customer_gvkey', 'year', 'quarter']],
                        left_on=['1_degree_customer', 'year', 'quarter'],
                        right_on=['firm_gvkey', 'year', 'quarter'],
                        how='left')
sc_link.drop('firm_gvkey_y', axis=1, inplace=True)
sc_link.rename({'firm_gvkey_x': 'firm_gvkey', 'customer_gvkey': '2_degree_customer'}, axis=1, inplace=True)
sc_link.drop_duplicates(inplace=True)
sc_link = sc_link.merge(customer[['firm_gvkey', 'customer_gvkey', 'year', 'quarter']],
                        left_on=['2_degree_customer', 'year', 'quarter'],
                        right_on=['firm_gvkey', 'year', 'quarter'],
                        how='left')
sc_link.drop('firm_gvkey_y', axis=1, inplace=True)
sc_link.rename({'firm_gvkey_x': 'firm_gvkey', 'customer_gvkey': '3_degree_customer'}, axis=1, inplace=True)
sc_link.drop_duplicates(inplace=True)

In [37]:
sc_link.loc[(sc_link['firm_gvkey'] == sc_link['1_degree_supplier']) | (sc_link['firm_gvkey'] == sc_link['2_degree_supplier']) | (sc_link['firm_gvkey'] == sc_link['3_degree_supplier']) | (sc_link['firm_gvkey'] == sc_link['1_degree_customer']) | (sc_link['firm_gvkey'] == sc_link['2_degree_customer']) | (sc_link['firm_gvkey'] == sc_link['3_degree_customer'])]

,firm_gvkey,year,quarter,1_degree_supplier,2_degree_supplier,3_degree_supplier,1_degree_customer,2_degree_customer,3_degree_customer
3287,31657.0,2019,2,29955.0,31657.0,31673.0,NaN,NaN,NaN
3288,31657.0,2019,2,29955.0,31657.0,29955.0,NaN,NaN,NaN
3308,31657.0,2019,3,29955.0,31657.0,31673.0,NaN,NaN,NaN
3309,31657.0,2019,3,29955.0,31657.0,29955.0,NaN,NaN,NaN
18494,2184.0,2019,4,1690.0,2184.0,160329.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
36207516,150937.0,2012,2,NaN,NaN,NaN,61591.0,12142.0,150937.0
36207554,150937.0,2012,3,NaN,NaN,NaN,61591.0,12142.0,150937.0
36207606,150937.0,2012,4,NaN,NaN,NaN,61591.0,12142.0,150937.0
36207687,150937.0,2013,1,NaN,NaN,NaN,61591.0,12142.0,150937.0


In [67]:
firm['firm_gvkey'].nunique()

4431

In [66]:
sc_link.groupby(['firm_gvkey', 'year', 'quarter'])['3_degree_supplier'].nunique().mean()

5.110244978278022

In [68]:
sc_link.to_pickle('./factset_revere_companies/fs_sc_link_3_degrees.pkl', compression='zip')

In [3]:
trucost = pd.read_pickle('./esg_scores/trucost.pkl', compression='zip')
trucost_wrds_companies = pd.read_csv('./esg_scores/trucost_wrds_companies.csv')

In [4]:
trucost = trucost.loc[(trucost['scoretype'] == 'S&P Global ESG Score') & (trucost['csascoretypename'] == 'Modeled')].drop_duplicates()
esg_scores = trucost[['scoredate', 'scorevalue', 'institutionid']].merge(trucost_wrds_companies[['gvkey', 'institutionid']],
                                                                    on='institutionid',
                                                                    how='left')
# trucost.drop('institutionid', axis=1, inplace=True)
esg_scores['scoredate'] = pd.to_datetime(esg_scores['scoredate'])
esg_scores['year'] = esg_scores.scoredate.dt.year
esg_scores['quarter'] = esg_scores.scoredate.dt.quarter
esg_scores.dropna(inplace=True)

In [101]:
firm = firm.merge(esg_scores,
                  left_on=['firm_gvkey', 'year', 'quarter'],
                  right_on=['gvkey', 'year', 'quarter'],
                  how='left')
firm.drop(['gvkey', 'institutionid', 'scoredate'], axis=1, inplace=True)

In [105]:
firm.sort_values(['firm_gvkey', 'year', 'quarter'], inplace=True)
firm['scorevalue'] = firm.groupby('firm_gvkey')['scorevalue'].fillna(method='ffill')

80479    NaN
80545    NaN
80546    NaN
80547    NaN
80548    NaN
          ..
100227   NaN
32893    NaN
32894    NaN
32895    NaN
32896    NaN
Name: scorevalue, Length: 105645, dtype: float64

In [107]:
firm.loc[(firm['ewfdsc_pol'].notna() | firm['ewfdcc_pol'].notna()) & (firm['ewsdcc_pol'].notna() | firm['ewsdsc_pol'].notna()) & (firm['ewtdcc_pol'].notna() | firm['ewtdsc_pol'].notna()) & (firm['scorevalue'].notna())]

,firm_gvkey,year,quarter,firm_pol,firm_covid,firm_cc,ewfdsc_pol,ewfdsc_covid,ewfdsc_cc,ewfdcc_pol,...,ewsdcc_pol,ewsdcc_covid,ewsdcc_cc,ewtdsc_pol,ewtdsc_covid,ewtdsc_cc,ewtdcc_pol,ewtdcc_covid,ewtdcc_cc,scorevalue
21476,1004.0,2019,4,19.646365,0.000000,0.000000,75.814716,0.000000,0.002201,64.349970,...,59.677054,0.000000,0.002048,72.064290,0.000000,0.003281,76.662307,0.000000,0.004018,24.0
21477,1004.0,2020,1,164.948454,1.255887,0.000441,94.698423,0.328148,0.001763,80.160585,...,66.783306,0.216774,0.002531,61.780371,0.619863,0.000813,66.152041,0.208594,0.002272,24.0
21469,1004.0,2020,3,111.798485,1.052449,0.000202,138.368021,1.436011,0.001702,133.075839,...,123.143480,1.662896,0.002563,102.987330,1.251230,0.000856,108.452708,1.789964,0.001893,24.0
21470,1004.0,2020,4,51.352277,0.389408,0.000000,87.492229,0.760750,0.002848,80.536957,...,103.812803,1.068126,0.001997,102.813022,1.031777,0.000760,91.261583,1.262975,0.002335,25.0
21471,1004.0,2021,1,54.161401,0.429092,0.001200,75.840333,0.617527,0.001550,82.798134,...,73.333769,1.100933,0.003029,30.703101,0.905797,0.000229,79.446555,1.185781,0.002760,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94231,328795.0,2021,4,57.247538,0.369959,0.007028,NaN,NaN,NaN,68.366051,...,92.563214,0.350416,0.015929,NaN,NaN,NaN,57.117285,0.235296,0.006280,29.0
17799,339965.0,2021,1,66.496731,0.123640,0.000333,49.820043,1.194801,0.000355,63.678146,...,67.955381,0.922006,0.000427,55.162849,1.023619,0.000483,81.036931,1.032483,0.001261,21.0
17800,339965.0,2021,2,36.518564,0.137174,0.000374,86.154861,0.914048,0.001183,89.058657,...,95.343278,1.212543,0.000405,80.948899,0.860770,0.001028,125.486389,1.219658,0.000505,21.0
17801,339965.0,2021,3,57.843591,0.259943,0.000337,78.402607,1.082985,0.001122,69.239550,...,57.325295,1.372581,0.000660,29.400978,0.518378,0.000902,111.338774,2.313784,0.001793,21.0


In [108]:
firm.loc[(firm['ewfdsc_pol'].notna() | firm['ewfdcc_pol'].notna()) & (firm['ewsdcc_pol'].notna() | firm['ewsdsc_pol'].notna()) & (firm['ewtdcc_pol'].notna() | firm['ewtdsc_pol'].notna()) & (firm['scorevalue'].notna()), 'firm_gvkey'].nunique()

1510

In [110]:
firm.loc[(firm['ewfdsc_pol'].notna() | firm['ewfdcc_pol'].notna()) & (firm['ewsdcc_pol'].notna() | firm['ewsdsc_pol'].notna()) & (firm['ewtdcc_pol'].notna() | firm['ewtdsc_pol'].notna())].groupby('year').size()

year
2006    1741
2007    2075
2008    2455
2009    2208
2010    2403
2011    3127
2012    3677
2013    4395
2014    4553
2015    4788
2016    5031
2017    6048
2018    6515
2019    6725
2020    6365
2021    6368
dtype: int64

In [111]:
firm.loc[(firm['ewfdsc_pol'].notna() | firm['ewfdcc_pol'].notna()) & (firm['ewsdcc_pol'].notna() | firm['ewsdsc_pol'].notna()) & (firm['ewtdcc_pol'].notna() | firm['ewtdsc_pol'].notna()) & (firm['scorevalue'].notna())].groupby('year').size()

year
2013     251
2014     737
2015     916
2016    1247
2017    1478
2018    1773
2019    2358
2020    3717
2021    4398
dtype: int64

In [2]:
sc_link = pd.read_pickle('./factset_revere_companies/fs_sc_link_3_degrees.pkl', compression='zip')

In [5]:
sc_link.head()

,firm_gvkey,year,quarter,1_degree_supplier,2_degree_supplier,3_degree_supplier,1_degree_customer,2_degree_customer,3_degree_customer
0,145046.0,2013,1,6066.0,12142.0,6008.0,NaN,NaN,NaN
1,145046.0,2013,1,6066.0,61591.0,174077.0,NaN,NaN,NaN
2,145046.0,2013,1,6066.0,61591.0,120774.0,NaN,NaN,NaN
3,145046.0,2013,1,6066.0,6008.0,63651.0,NaN,NaN,NaN
4,145046.0,2013,1,6066.0,6008.0,1704.0,NaN,NaN,NaN


In [41]:
third_degree_supplier = sc_link[['firm_gvkey', 'year', 'quarter', '3_degree_supplier']].drop_duplicates().dropna()
third_degree_supplier['lst_yr'] = third_degree_supplier['year'] - 1
third_degree_supplier = third_degree_supplier.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_supplier', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
third_degree_supplier.drop('gvkey', axis=1, inplace=True)
third_degree_supplier.drop_duplicates(inplace=True)
third_degree_supplier.sort_values(['firm_gvkey', 'year', 'quarter', '3_degree_supplier'], inplace=True)
third_degree_supplier['scorevalue'] = third_degree_supplier.groupby(['firm_gvkey', '3_degree_supplier'])['scorevalue'].fillna(method='ffill')
third_degree_supplier = third_degree_supplier.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_supplier', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
third_degree_supplier.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
third_degree_supplier.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
third_degree_supplier.drop_duplicates(inplace=True)
third_degree_supplier['lst_score'] = third_degree_supplier.groupby(['firm_gvkey', '3_degree_supplier'])['lst_score'].fillna(method='ffill')
third_degree_supplier['esg_score_decline'] = np.where(third_degree_supplier['scorevalue'] < third_degree_supplier['lst_score'], 1, 0)

In [47]:
third_degree_supplier.dropna()

,firm_gvkey,year,quarter,3_degree_supplier,scorevalue,lst_score,esg_score_decline
70,1004.0,2017,3,1300.0,45.0,44.0,0
71,1004.0,2017,3,1773.0,37.0,36.0,0
73,1004.0,2017,3,2285.0,52.0,48.0,0
76,1004.0,2017,3,4199.0,57.0,57.0,0
78,1004.0,2017,3,5047.0,58.0,56.0,0
...,...,...,...,...,...,...,...
539324,345980.0,2021,4,61676.0,39.0,34.0,0
539325,345980.0,2021,4,61676.0,41.0,34.0,0
539326,345980.0,2021,4,64768.0,40.0,37.0,0
539327,345980.0,2021,4,64853.0,32.0,29.0,0


In [80]:
third_degree_supplier.dropna()['firm_gvkey'].nunique()

1778

In [81]:
third_degree_supplier.dropna()['3_degree_supplier'].nunique()

604

In [124]:
third_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].mean().mean()

0.48580001333843953

In [101]:
third_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].std().mean()

0.33720838394842517

In [120]:
third_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.25).mean()

0.2

In [121]:
third_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.75).mean()

0.6923076923076923

In [122]:
third_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].median().mean()

0.5

In [46]:
second_degree_supplier = sc_link[['firm_gvkey', 'year', 'quarter', '2_degree_supplier']].drop_duplicates().dropna()
second_degree_supplier['lst_yr'] = second_degree_supplier['year'] - 1
second_degree_supplier = second_degree_supplier.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['2_degree_supplier', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
second_degree_supplier.drop('gvkey', axis=1, inplace=True)
second_degree_supplier.drop_duplicates(inplace=True)
second_degree_supplier.sort_values(['firm_gvkey', 'year', 'quarter', '2_degree_supplier'], inplace=True)
second_degree_supplier['scorevalue'] = second_degree_supplier.groupby(['firm_gvkey', '2_degree_supplier'])['scorevalue'].fillna(method='ffill')
second_degree_supplier = second_degree_supplier.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['2_degree_supplier', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
second_degree_supplier.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
second_degree_supplier.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
second_degree_supplier.drop_duplicates(inplace=True)
second_degree_supplier['lst_score'] = second_degree_supplier.groupby(['firm_gvkey', '2_degree_supplier'])['lst_score'].fillna(method='ffill')
second_degree_supplier['esg_score_decline'] = np.where(second_degree_supplier['scorevalue'] < second_degree_supplier['lst_score'], 1, 0)

In [60]:
second_degree_supplier.dropna()

,firm_gvkey,year,quarter,2_degree_supplier,scorevalue,lst_score,esg_score_decline
20,1004.0,2017,3,2285.0,52.0,48.0,0
22,1004.0,2017,3,10519.0,47.0,44.0,0
23,1004.0,2017,4,2285.0,52.0,48.0,0
25,1004.0,2017,4,10519.0,47.0,44.0,0
26,1004.0,2018,1,2285.0,52.0,48.0,0
...,...,...,...,...,...,...,...
296972,345980.0,2021,4,61676.0,39.0,34.0,0
296973,345980.0,2021,4,61676.0,41.0,34.0,0
296974,345980.0,2021,4,64853.0,32.0,29.0,0
296975,345980.0,2021,4,64853.0,33.0,29.0,0


In [77]:
second_degree_supplier.dropna()['firm_gvkey'].nunique()

1811

In [78]:
second_degree_supplier.dropna()['2_degree_supplier'].nunique()

718

In [79]:
second_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].mean().mean()

0.4784705344459565

In [100]:
second_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].std().mean()

0.38186351416817405

In [117]:
second_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.25).mean()

0.0

In [118]:
second_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.75).mean()

0.875

In [119]:
second_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].median().mean()

0.5

In [55]:
first_degree_supplier = sc_link[['firm_gvkey', 'year', 'quarter', '1_degree_supplier']].drop_duplicates().dropna()
first_degree_supplier['lst_yr'] = first_degree_supplier['year'] - 1
first_degree_supplier = first_degree_supplier.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['1_degree_supplier', 'year', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left')
first_degree_supplier.drop('gvkey', axis=1, inplace=True)
first_degree_supplier.drop_duplicates(inplace=True)
first_degree_supplier.sort_values(['firm_gvkey', 'year', 'quarter', '1_degree_supplier'], inplace=True)
first_degree_supplier['scorevalue'] = first_degree_supplier.groupby(['firm_gvkey', '1_degree_supplier'])['scorevalue'].fillna(method='ffill')
first_degree_supplier = first_degree_supplier.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['1_degree_supplier', 'lst_yr', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left',
                                                      suffixes=('', '_1'))
first_degree_supplier.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
first_degree_supplier.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
first_degree_supplier.drop_duplicates(inplace=True)
first_degree_supplier['lst_score'] = first_degree_supplier.groupby(['firm_gvkey', '1_degree_supplier'])['lst_score'].fillna(method='ffill')
first_degree_supplier['esg_score_decline'] = np.where(first_degree_supplier['scorevalue'] < first_degree_supplier['lst_score'], 1, 0)

In [59]:
first_degree_supplier.dropna()

,firm_gvkey,year,quarter,1_degree_supplier,scorevalue,lst_score,esg_score_decline
12,1004.0,2016,3,5047.0,56.0,67.0,1
13,1004.0,2016,4,5047.0,56.0,67.0,1
14,1004.0,2017,1,5047.0,56.0,67.0,1
15,1004.0,2017,3,5047.0,58.0,56.0,0
16,1004.0,2017,4,5047.0,58.0,56.0,0
...,...,...,...,...,...,...,...
159971,345980.0,2021,2,170617.0,31.0,34.0,1
159976,345980.0,2021,3,170617.0,31.0,34.0,1
159977,345980.0,2021,4,1690.0,49.0,47.0,0
159978,345980.0,2021,4,2136.0,49.0,47.0,0


In [75]:
first_degree_supplier.dropna()['firm_gvkey'].nunique()

1909

In [74]:
first_degree_supplier.dropna()['1_degree_supplier'].nunique()

969

In [99]:
first_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].mean().mean()

0.4471290115594896

In [98]:
first_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].std().mean()

0.43330287132115525

In [114]:
first_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.25).mean()

0.0

In [115]:
first_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.75).mean()

1.0

In [116]:
first_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].median().mean()

0.4

In [61]:
third_degree_customer = sc_link[['firm_gvkey', 'year', 'quarter', '3_degree_customer']].drop_duplicates().dropna()
third_degree_customer['lst_yr'] = third_degree_customer['year'] - 1
third_degree_customer = third_degree_customer.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_customer', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
third_degree_customer.drop('gvkey', axis=1, inplace=True)
third_degree_customer.drop_duplicates(inplace=True)
third_degree_customer.sort_values(['firm_gvkey', 'year', 'quarter', '3_degree_customer'], inplace=True)
third_degree_customer['scorevalue'] = third_degree_customer.groupby(['firm_gvkey', '3_degree_customer'])['scorevalue'].fillna(method='ffill')
third_degree_customer = third_degree_customer.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_customer', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
third_degree_customer.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
third_degree_customer.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
third_degree_customer.drop_duplicates(inplace=True)
third_degree_customer['lst_score'] = third_degree_customer.groupby(['firm_gvkey', '3_degree_customer'])['lst_score'].fillna(method='ffill')
third_degree_customer['esg_score_decline'] = np.where(third_degree_customer['scorevalue'] < third_degree_customer['lst_score'], 1, 0)

In [64]:
third_degree_customer.dropna()

,firm_gvkey,year,quarter,3_degree_customer,scorevalue,lst_score,esg_score_decline
187,1004.0,2014,3,6730.0,52.0,54.0,1
188,1004.0,2014,3,7063.0,40.0,42.0,1
189,1004.0,2014,3,7085.0,48.0,54.0,1
191,1004.0,2014,3,7171.0,46.0,47.0,1
192,1004.0,2014,3,7257.0,64.0,67.0,1
...,...,...,...,...,...,...,...
1606543,339965.0,2021,1,181104.0,33.0,37.0,1
1606545,339965.0,2021,1,186159.0,30.0,28.0,0
1606546,339965.0,2021,1,191255.0,32.0,34.0,1
1606547,339965.0,2021,1,264265.0,34.0,38.0,1


In [89]:
third_degree_customer.dropna()['firm_gvkey'].nunique()

2144

In [90]:
third_degree_customer.dropna()['3_degree_customer'].nunique()

909

In [96]:
third_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].mean().mean()

0.5050957753636479

In [95]:
third_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].std().mean()

0.29723744256787166

In [111]:
third_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.25).mean()

0.3

In [112]:
third_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.75).mean()

0.7058823529411765

In [113]:
third_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].median().mean()

0.5

In [66]:
second_degree_customer = sc_link[['firm_gvkey', 'year', 'quarter', '2_degree_customer']].drop_duplicates().dropna()
second_degree_customer['lst_yr'] = second_degree_customer['year'] - 1
second_degree_customer = second_degree_customer.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['2_degree_customer', 'year', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left')
second_degree_customer.drop('gvkey', axis=1, inplace=True)
second_degree_customer.drop_duplicates(inplace=True)
second_degree_customer.sort_values(['firm_gvkey', 'year', 'quarter', '2_degree_customer'], inplace=True)
second_degree_customer['scorevalue'] = second_degree_customer.groupby(['firm_gvkey', '2_degree_customer'])['scorevalue'].fillna(method='ffill')
second_degree_customer = second_degree_customer.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['2_degree_customer', 'lst_yr', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left',
                                                      suffixes=('', '_1'))
second_degree_customer.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
second_degree_customer.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
second_degree_customer.drop_duplicates(inplace=True)
second_degree_customer['lst_score'] = second_degree_customer.groupby(['firm_gvkey', '2_degree_customer'])['lst_score'].fillna(method='ffill')
second_degree_customer['esg_score_decline'] = np.where(second_degree_customer['scorevalue'] < second_degree_customer['lst_score'], 1, 0)

In [68]:
second_degree_customer.dropna()

,firm_gvkey,year,quarter,2_degree_customer,scorevalue,lst_score,esg_score_decline
131,1004.0,2014,3,2285.0,49.0,53.0,1
135,1004.0,2014,3,4029.0,51.0,55.0,1
141,1004.0,2014,3,7257.0,64.0,67.0,1
142,1004.0,2014,3,7620.0,42.0,45.0,1
144,1004.0,2014,3,8455.0,42.0,47.0,1
...,...,...,...,...,...,...,...
815974,339965.0,2021,1,145049.0,23.0,27.0,1
815975,339965.0,2021,1,149177.0,24.0,25.0,1
815983,339965.0,2021,1,190455.0,25.0,27.0,1
816003,339965.0,2021,4,1487.0,42.0,30.0,0


In [86]:
second_degree_customer.dropna()['firm_gvkey'].nunique()

2178

In [87]:
second_degree_customer.dropna()['2_degree_customer'].nunique()

1009

In [88]:
second_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].mean().mean()

0.4867602802210595

In [103]:
second_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].std().mean()

0.3366212912123355

In [106]:
second_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.25).mean()

0.2222222222222222

In [107]:
second_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.75).mean()

0.7368421052631579

In [110]:
second_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].median().mean()

0.5

In [69]:
first_degree_customer = sc_link[['firm_gvkey', 'year', 'quarter', '1_degree_customer']].drop_duplicates().dropna()
first_degree_customer['lst_yr'] = first_degree_customer['year'] - 1
first_degree_customer = first_degree_customer.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['1_degree_customer', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
first_degree_customer.drop('gvkey', axis=1, inplace=True)
first_degree_customer.drop_duplicates(inplace=True)
first_degree_customer.sort_values(['firm_gvkey', 'year', 'quarter', '1_degree_customer'], inplace=True)
first_degree_customer['scorevalue'] = first_degree_customer.groupby(['firm_gvkey', '1_degree_customer'])['scorevalue'].fillna(method='ffill')
first_degree_customer = first_degree_customer.merge(esg_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['1_degree_customer', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
first_degree_customer.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
first_degree_customer.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
first_degree_customer.drop_duplicates(inplace=True)
first_degree_customer['lst_score'] = first_degree_customer.groupby(['firm_gvkey', '1_degree_customer'])['lst_score'].fillna(method='ffill')
first_degree_customer['esg_score_decline'] = np.where(first_degree_customer['scorevalue'] < first_degree_customer['lst_score'], 1, 0)

In [70]:
first_degree_customer.dropna()

,firm_gvkey,year,quarter,1_degree_customer,scorevalue,lst_score,esg_score_decline
69,1004.0,2015,1,10795.0,52.0,52.0,0
71,1004.0,2015,3,10795.0,47.0,52.0,1
73,1004.0,2015,4,10795.0,47.0,52.0,1
75,1004.0,2016,1,10795.0,47.0,52.0,1
78,1004.0,2016,3,1300.0,44.0,45.0,1
...,...,...,...,...,...,...,...
393568,339965.0,2021,1,63172.0,37.0,35.0,0
393578,339965.0,2021,2,63172.0,37.0,35.0,0
393587,339965.0,2021,3,63172.0,37.0,35.0,0
393592,339965.0,2021,4,7171.0,45.0,45.0,0


In [83]:
first_degree_customer.dropna()['firm_gvkey'].nunique()

2378

In [84]:
first_degree_customer.dropna()['1_degree_customer'].nunique()

1312

In [85]:
first_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].mean().mean()

0.47598460104023615

In [102]:
first_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].std().mean()

0.38784594056654526

In [104]:
first_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.25).mean()

0.0

In [105]:
first_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].quantile(0.75).mean()

1.0

In [109]:
first_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].apply(lambda x: x.sum() / len(x)).reset_index().groupby(['year', 'quarter'])['esg_score_decline'].median().mean()

0.5

In [130]:
first_degree_supplier = first_degree_supplier.dropna()
second_degree_supplier = second_degree_supplier.dropna()
third_degree_supplier = third_degree_supplier.dropna()
first_degree_customer = first_degree_customer.dropna()
second_degree_customer = second_degree_customer.dropna()
third_degree_customer = third_degree_customer.dropna()
first_degree_supplier['esg_score_decline_rate'] = first_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
second_degree_supplier['esg_score_decline_rate'] = second_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
third_degree_supplier['esg_score_decline_rate'] = third_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
first_degree_customer['esg_score_decline_rate'] = first_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
second_degree_customer['esg_score_decline_rate'] = second_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
third_degree_customer['esg_score_decline_rate'] = third_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))

In [149]:
second_degree_customer.groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).sum()/30362

0.4106678720759408

In [151]:
first_degree_supplier.to_csv('./factset_revere_companies/ESG_scores/1-s.csv')
second_degree_supplier.to_csv('./factset_revere_companies/ESG_scores/2-s.csv')
third_degree_supplier.to_csv('./factset_revere_companies/ESG_scores/3-s.csv')
first_degree_customer.to_csv('./factset_revere_companies/ESG_scores/1-c.csv')
second_degree_customer.to_csv('./factset_revere_companies/ESG_scores/2-c.csv')
third_degree_customer.to_csv('./factset_revere_companies/ESG_scores/3-c.csv')
first_degree_supplier.groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/ESG_scores/1-s-firm.csv')
second_degree_supplier.groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/ESG_scores/2-s-firm.csv')
third_degree_supplier.groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/ESG_scores/3-s-firm.csv')
first_degree_customer.groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/ESG_scores/1-c-firm.csv')
second_degree_customer.groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/ESG_scores/2-c-firm.csv')
third_degree_customer.groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/ESG_scores/3-c-firm.csv')

In [2]:
trucost = pd.read_pickle('./esg_scores/trucost.pkl', compression='zip')
trucost_wrds_companies = pd.read_csv('./esg_scores/trucost_wrds_companies.csv')

In [4]:
trucost = trucost.loc[(trucost['dimensionname'] == 'Environmental Dimension') & (trucost['csascoretypename'] == 'Modeled') & (trucost['scoretype'] == 'Dimension') & (trucost['aspectname'] == 'Environmental Dimension') & (trucost['mostrecentsequence'] == 1)].drop_duplicates()
environmental_scores = trucost[['scoredate', 'scorevalue', 'institutionid']].merge(trucost_wrds_companies[['gvkey', 'institutionid']],
                                                                         on='institutionid',
                                                                         how='left')
# trucost.drop('institutionid', axis=1, inplace=True)
environmental_scores['scoredate'] = pd.to_datetime(environmental_scores['scoredate'])
environmental_scores['year'] = environmental_scores.scoredate.dt.year
environmental_scores['quarter'] = environmental_scores.scoredate.dt.quarter
environmental_scores.dropna(inplace=True)

In [5]:
third_degree_supplier = sc_link[['firm_gvkey', 'year', 'quarter', '3_degree_supplier']].drop_duplicates().dropna()
third_degree_supplier['lst_yr'] = third_degree_supplier['year'] - 1
third_degree_supplier = third_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_supplier', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
third_degree_supplier.drop('gvkey', axis=1, inplace=True)
third_degree_supplier.drop_duplicates(inplace=True)
third_degree_supplier.sort_values(['firm_gvkey', 'year', 'quarter', '3_degree_supplier'], inplace=True)
third_degree_supplier['scorevalue'] = third_degree_supplier.groupby(['firm_gvkey', '3_degree_supplier'])['scorevalue'].fillna(method='ffill')
third_degree_supplier = third_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_supplier', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
third_degree_supplier.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
third_degree_supplier.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
third_degree_supplier.drop_duplicates(inplace=True)
third_degree_supplier['lst_score'] = third_degree_supplier.groupby(['firm_gvkey', '3_degree_supplier'])['lst_score'].fillna(method='ffill')
third_degree_supplier['esg_score_decline'] = np.where(third_degree_supplier['scorevalue'] < third_degree_supplier['lst_score'], 1, 0)
second_degree_supplier = sc_link[['firm_gvkey', 'year', 'quarter', '2_degree_supplier']].drop_duplicates().dropna()
second_degree_supplier['lst_yr'] = second_degree_supplier['year'] - 1
second_degree_supplier = second_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['2_degree_supplier', 'year', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left')
second_degree_supplier.drop('gvkey', axis=1, inplace=True)
second_degree_supplier.drop_duplicates(inplace=True)
second_degree_supplier.sort_values(['firm_gvkey', 'year', 'quarter', '2_degree_supplier'], inplace=True)
second_degree_supplier['scorevalue'] = second_degree_supplier.groupby(['firm_gvkey', '2_degree_supplier'])['scorevalue'].fillna(method='ffill')
second_degree_supplier = second_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['2_degree_supplier', 'lst_yr', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left',
                                                      suffixes=('', '_1'))
second_degree_supplier.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
second_degree_supplier.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
second_degree_supplier.drop_duplicates(inplace=True)
second_degree_supplier['lst_score'] = second_degree_supplier.groupby(['firm_gvkey', '2_degree_supplier'])['lst_score'].fillna(method='ffill')
second_degree_supplier['esg_score_decline'] = np.where(second_degree_supplier['scorevalue'] < second_degree_supplier['lst_score'], 1, 0)
first_degree_supplier = sc_link[['firm_gvkey', 'year', 'quarter', '1_degree_supplier']].drop_duplicates().dropna()
first_degree_supplier['lst_yr'] = first_degree_supplier['year'] - 1
first_degree_supplier = first_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['1_degree_supplier', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
first_degree_supplier.drop('gvkey', axis=1, inplace=True)
first_degree_supplier.drop_duplicates(inplace=True)
first_degree_supplier.sort_values(['firm_gvkey', 'year', 'quarter', '1_degree_supplier'], inplace=True)
first_degree_supplier['scorevalue'] = first_degree_supplier.groupby(['firm_gvkey', '1_degree_supplier'])['scorevalue'].fillna(method='ffill')
first_degree_supplier = first_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['1_degree_supplier', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
first_degree_supplier.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
first_degree_supplier.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
first_degree_supplier.drop_duplicates(inplace=True)
first_degree_supplier['lst_score'] = first_degree_supplier.groupby(['firm_gvkey', '1_degree_supplier'])['lst_score'].fillna(method='ffill')
first_degree_supplier['esg_score_decline'] = np.where(first_degree_supplier['scorevalue'] < first_degree_supplier['lst_score'], 1, 0)

In [6]:
third_degree_customer = sc_link[['firm_gvkey', 'year', 'quarter', '3_degree_customer']].drop_duplicates().dropna()
third_degree_customer['lst_yr'] = third_degree_customer['year'] - 1
third_degree_customer = third_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_customer', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
third_degree_customer.drop('gvkey', axis=1, inplace=True)
third_degree_customer.drop_duplicates(inplace=True)
third_degree_customer.sort_values(['firm_gvkey', 'year', 'quarter', '3_degree_customer'], inplace=True)
third_degree_customer['scorevalue'] = third_degree_customer.groupby(['firm_gvkey', '3_degree_customer'])['scorevalue'].fillna(method='ffill')
third_degree_customer = third_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['3_degree_customer', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
third_degree_customer.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
third_degree_customer.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
third_degree_customer.drop_duplicates(inplace=True)
third_degree_customer['lst_score'] = third_degree_customer.groupby(['firm_gvkey', '3_degree_customer'])['lst_score'].fillna(method='ffill')
third_degree_customer['esg_score_decline'] = np.where(third_degree_customer['scorevalue'] < third_degree_customer['lst_score'], 1, 0)
second_degree_customer = sc_link[['firm_gvkey', 'year', 'quarter', '2_degree_customer']].drop_duplicates().dropna()
second_degree_customer['lst_yr'] = second_degree_customer['year'] - 1
second_degree_customer = second_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['2_degree_customer', 'year', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left')
second_degree_customer.drop('gvkey', axis=1, inplace=True)
second_degree_customer.drop_duplicates(inplace=True)
second_degree_customer.sort_values(['firm_gvkey', 'year', 'quarter', '2_degree_customer'], inplace=True)
second_degree_customer['scorevalue'] = second_degree_customer.groupby(['firm_gvkey', '2_degree_customer'])['scorevalue'].fillna(method='ffill')
second_degree_customer = second_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                      left_on=['2_degree_customer', 'lst_yr', 'quarter'],
                                                      right_on=['gvkey', 'year', 'quarter'],
                                                      how='left',
                                                      suffixes=('', '_1'))
second_degree_customer.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
second_degree_customer.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
second_degree_customer.drop_duplicates(inplace=True)
second_degree_customer['lst_score'] = second_degree_customer.groupby(['firm_gvkey', '2_degree_customer'])['lst_score'].fillna(method='ffill')
second_degree_customer['esg_score_decline'] = np.where(second_degree_customer['scorevalue'] < second_degree_customer['lst_score'], 1, 0)
first_degree_customer = sc_link[['firm_gvkey', 'year', 'quarter', '1_degree_customer']].drop_duplicates().dropna()
first_degree_customer['lst_yr'] = first_degree_customer['year'] - 1
first_degree_customer = first_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['1_degree_customer', 'year', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left')
first_degree_customer.drop('gvkey', axis=1, inplace=True)
first_degree_customer.drop_duplicates(inplace=True)
first_degree_customer.sort_values(['firm_gvkey', 'year', 'quarter', '1_degree_customer'], inplace=True)
first_degree_customer['scorevalue'] = first_degree_customer.groupby(['firm_gvkey', '1_degree_customer'])['scorevalue'].fillna(method='ffill')
first_degree_customer = first_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'year', 'quarter']],
                                                    left_on=['1_degree_customer', 'lst_yr', 'quarter'],
                                                    right_on=['gvkey', 'year', 'quarter'],
                                                    how='left',
                                                    suffixes=('', '_1'))
first_degree_customer.rename({'scorevalue_1': 'lst_score'}, axis=1, inplace=True)
first_degree_customer.drop(['lst_yr', 'year_1', 'gvkey'], axis=1, inplace=True)
first_degree_customer.drop_duplicates(inplace=True)
first_degree_customer['lst_score'] = first_degree_customer.groupby(['firm_gvkey', '1_degree_customer'])['lst_score'].fillna(method='ffill')
first_degree_customer['esg_score_decline'] = np.where(first_degree_customer['scorevalue'] < first_degree_customer['lst_score'], 1, 0)

In [7]:
# quarter
first_degree_supplier = first_degree_supplier.dropna()
second_degree_supplier = second_degree_supplier.dropna()
third_degree_supplier = third_degree_supplier.dropna()
first_degree_customer = first_degree_customer.dropna()
second_degree_customer = second_degree_customer.dropna()
third_degree_customer = third_degree_customer.dropna()
first_degree_supplier['environmental_score_decline_rate'] = first_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
second_degree_supplier['environmental_score_decline_rate'] = second_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
third_degree_supplier['environmental_score_decline_rate'] = third_degree_supplier.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
first_degree_customer['environmental_score_decline_rate'] = first_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
second_degree_customer['environmental_score_decline_rate'] = second_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
third_degree_customer['environmental_score_decline_rate'] = third_degree_customer.dropna().groupby(['firm_gvkey', 'year', 'quarter'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))

In [46]:
first_degree_supplier.loc[(first_degree_supplier['firm_gvkey'] == 1004)].dropna()

,firm_gvkey,year,1_degree_supplier,scorevalue,lst_score,esg_score_decline,environmental_score_decline_rate
19530,1004.0,2016,5047.0,59.0,66.0,1,1.000000
19531,1004.0,2017,5047.0,61.0,59.0,0,0.000000
19525,1004.0,2018,5047.0,62.0,61.0,0,0.000000
19520,1004.0,2019,1300.0,44.0,42.0,0,0.333333
19519,1004.0,2019,1598.0,31.0,27.0,0,0.333333
19521,1004.0,2019,5047.0,53.0,62.0,1,0.333333
19505,1004.0,2020,1300.0,44.0,44.0,0,0.000000
19504,1004.0,2020,1598.0,31.0,31.0,0,0.000000
19507,1004.0,2020,4199.0,59.0,48.0,0,0.000000
19506,1004.0,2020,5047.0,55.0,53.0,0,0.000000


In [8]:
first_degree_supplier.to_csv('./factset_revere_companies/Environmental_Dimension/quarter/1-s.csv')
second_degree_supplier.to_csv('./factset_revere_companies/Environmental_Dimension/quarter/2-s.csv')
third_degree_supplier.to_csv('./factset_revere_companies/Environmental_Dimension/quarter/3-s.csv')
first_degree_customer.to_csv('./factset_revere_companies/Environmental_Dimension/quarter/1-c.csv')
second_degree_customer.to_csv('./factset_revere_companies/Environmental_Dimension/quarter/2-c.csv')
third_degree_customer.to_csv('./factset_revere_companies/Environmental_Dimension/quarter/3-c.csv')
first_degree_supplier.groupby(['firm_gvkey', 'year', 'quarter'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/quarter/1-s-firm.csv')
second_degree_supplier.groupby(['firm_gvkey', 'year', 'quarter'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/quarter/2-s-firm.csv')
third_degree_supplier.groupby(['firm_gvkey', 'year', 'quarter'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/quarter/3-s-firm.csv')
first_degree_customer.groupby(['firm_gvkey', 'year', 'quarter'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/quarter/1-c-firm.csv')
second_degree_customer.groupby(['firm_gvkey', 'year', 'quarter'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/quarter/2-c-firm.csv')
third_degree_customer.groupby(['firm_gvkey', 'year', 'quarter'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/quarter/3-c-firm.csv')

In [34]:
# year
trucost = trucost.loc[(trucost['dimensionname'] == 'Environmental Dimension') & (trucost['csascoretypename'] == 'Modeled') & (trucost['scoretype'] == 'Dimension') & (trucost['aspectname'] == 'Environmental Dimension') & (trucost['mostrecentsequence'] == 1)].drop_duplicates()
environmental_scores = trucost[['assessmentyear', 'scorevalue', 'institutionid']].merge(trucost_wrds_companies[['gvkey', 'institutionid']],
                                                                                   on='institutionid',
                                                                                   how='left')
environmental_scores.dropna(inplace=True)

In [40]:
third_degree_supplier = sc_link[['firm_gvkey', 'year', '3_degree_supplier']].drop_duplicates().dropna()
third_degree_supplier = third_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'assessmentyear']],
                                                    left_on=['3_degree_supplier', 'year'],
                                                    right_on=['gvkey', 'assessmentyear'],
                                                    how='left')
third_degree_supplier.drop(['gvkey', 'assessmentyear'], axis=1, inplace=True)
third_degree_supplier.drop_duplicates(inplace=True)
third_degree_supplier.sort_values(['firm_gvkey', 'year', '3_degree_supplier'], inplace=True)
third_degree_supplier['scorevalue'] = third_degree_supplier.groupby(['firm_gvkey', '3_degree_supplier'])['scorevalue'].fillna(method='ffill')
third_degree_supplier['lst_score'] = third_degree_supplier.groupby(['firm_gvkey', '3_degree_supplier'])['scorevalue'].shift(1)
third_degree_supplier['esg_score_decline'] = np.where(third_degree_supplier['scorevalue'] < third_degree_supplier['lst_score'], 1, 0)
second_degree_supplier = sc_link[['firm_gvkey', 'year', '2_degree_supplier']].drop_duplicates().dropna()
second_degree_supplier = second_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'assessmentyear']],
                                                    left_on=['2_degree_supplier', 'year'],
                                                    right_on=['gvkey', 'assessmentyear'],
                                                    how='left')
second_degree_supplier.drop(['gvkey', 'assessmentyear'], axis=1, inplace=True)
second_degree_supplier.drop_duplicates(inplace=True)
second_degree_supplier.sort_values(['firm_gvkey', 'year', '2_degree_supplier'], inplace=True)
second_degree_supplier['scorevalue'] = second_degree_supplier.groupby(['firm_gvkey', '2_degree_supplier'])['scorevalue'].fillna(method='ffill')
second_degree_supplier['lst_score'] = second_degree_supplier.groupby(['firm_gvkey', '2_degree_supplier'])['scorevalue'].shift(1)
second_degree_supplier['esg_score_decline'] = np.where(second_degree_supplier['scorevalue'] < second_degree_supplier['lst_score'], 1, 0)
first_degree_supplier = sc_link[['firm_gvkey', 'year', '1_degree_supplier']].drop_duplicates().dropna()
first_degree_supplier = first_degree_supplier.merge(environmental_scores[['scorevalue', 'gvkey', 'assessmentyear']],
                                                      left_on=['1_degree_supplier', 'year'],
                                                      right_on=['gvkey', 'assessmentyear'],
                                                      how='left')
first_degree_supplier.drop(['gvkey', 'assessmentyear'], axis=1, inplace=True)
first_degree_supplier.drop_duplicates(inplace=True)
first_degree_supplier.sort_values(['firm_gvkey', 'year', '1_degree_supplier'], inplace=True)
first_degree_supplier['scorevalue'] = first_degree_supplier.groupby(['firm_gvkey', '1_degree_supplier'])['scorevalue'].fillna(method='ffill')
first_degree_supplier['lst_score'] = first_degree_supplier.groupby(['firm_gvkey', '1_degree_supplier'])['scorevalue'].shift(1)
first_degree_supplier['esg_score_decline'] = np.where(first_degree_supplier['scorevalue'] < first_degree_supplier['lst_score'], 1, 0)

In [49]:
third_degree_customer = sc_link[['firm_gvkey', 'year', '3_degree_customer']].drop_duplicates().dropna()
third_degree_customer = third_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'assessmentyear']],
                                                    left_on=['3_degree_customer', 'year'],
                                                    right_on=['gvkey', 'assessmentyear'],
                                                    how='left')
third_degree_customer.drop(['gvkey', 'assessmentyear'], axis=1, inplace=True)
third_degree_customer.drop_duplicates(inplace=True)
third_degree_customer.sort_values(['firm_gvkey', 'year', '3_degree_customer'], inplace=True)
third_degree_customer['scorevalue'] = third_degree_customer.groupby(['firm_gvkey', '3_degree_customer'])['scorevalue'].fillna(method='ffill')
third_degree_customer['lst_score'] = third_degree_customer.groupby(['firm_gvkey', '3_degree_customer'])['scorevalue'].shift(1)
third_degree_customer['esg_score_decline'] = np.where(third_degree_customer['scorevalue'] < third_degree_customer['lst_score'], 1, 0)
second_degree_customer = sc_link[['firm_gvkey', 'year', '2_degree_customer']].drop_duplicates().dropna()
second_degree_customer = second_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'assessmentyear']],
                                                      left_on=['2_degree_customer', 'year'],
                                                      right_on=['gvkey', 'assessmentyear'],
                                                      how='left')
second_degree_customer.drop(['gvkey', 'assessmentyear'], axis=1, inplace=True)
second_degree_customer.drop_duplicates(inplace=True)
second_degree_customer.sort_values(['firm_gvkey', 'year', '2_degree_customer'], inplace=True)
second_degree_customer['scorevalue'] = second_degree_customer.groupby(['firm_gvkey', '2_degree_customer'])['scorevalue'].fillna(method='ffill')
second_degree_customer['lst_score'] = second_degree_customer.groupby(['firm_gvkey', '2_degree_customer'])['scorevalue'].shift(1)
second_degree_customer['esg_score_decline'] = np.where(second_degree_customer['scorevalue'] < second_degree_customer['lst_score'], 1, 0)
first_degree_customer = sc_link[['firm_gvkey', 'year', '1_degree_customer']].drop_duplicates().dropna()
first_degree_customer = first_degree_customer.merge(environmental_scores[['scorevalue', 'gvkey', 'assessmentyear']],
                                                    left_on=['1_degree_customer', 'year'],
                                                    right_on=['gvkey', 'assessmentyear'],
                                                    how='left')
first_degree_customer.drop(['gvkey', 'assessmentyear'], axis=1, inplace=True)
first_degree_customer.drop_duplicates(inplace=True)
first_degree_customer.sort_values(['firm_gvkey', 'year', '1_degree_customer'], inplace=True)
first_degree_customer['scorevalue'] = first_degree_customer.groupby(['firm_gvkey', '1_degree_customer'])['scorevalue'].fillna(method='ffill')
first_degree_customer['lst_score'] = first_degree_customer.groupby(['firm_gvkey', '1_degree_customer'])['scorevalue'].shift(1)
first_degree_customer['esg_score_decline'] = np.where(first_degree_customer['scorevalue'] < first_degree_customer['lst_score'], 1, 0)

In [50]:
first_degree_supplier = first_degree_supplier.dropna()
second_degree_supplier = second_degree_supplier.dropna()
third_degree_supplier = third_degree_supplier.dropna()
first_degree_customer = first_degree_customer.dropna()
second_degree_customer = second_degree_customer.dropna()
third_degree_customer = third_degree_customer.dropna()

In [51]:
first_degree_supplier['environmental_score_decline_rate'] = first_degree_supplier.dropna().groupby(['firm_gvkey', 'year'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
second_degree_supplier['environmental_score_decline_rate'] = second_degree_supplier.dropna().groupby(['firm_gvkey', 'year'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
third_degree_supplier['environmental_score_decline_rate'] = third_degree_supplier.dropna().groupby(['firm_gvkey', 'year'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
first_degree_customer['environmental_score_decline_rate'] = first_degree_customer.dropna().groupby(['firm_gvkey', 'year'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
second_degree_customer['environmental_score_decline_rate'] = second_degree_customer.dropna().groupby(['firm_gvkey', 'year'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))
third_degree_customer['environmental_score_decline_rate'] = third_degree_customer.dropna().groupby(['firm_gvkey', 'year'])['esg_score_decline'].transform(lambda x: x.sum() / len(x))

In [54]:
first_degree_supplier.groupby(['firm_gvkey', 'year'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/1-s-firm.csv')
second_degree_supplier.groupby(['firm_gvkey', 'year'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/2-s-firm.csv')
third_degree_supplier.groupby(['firm_gvkey', 'year'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/3-s-firm.csv')
first_degree_customer.groupby(['firm_gvkey', 'year'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/1-c-firm.csv')
second_degree_customer.groupby(['firm_gvkey', 'year'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/2-c-firm.csv')
third_degree_customer.groupby(['firm_gvkey', 'year'])['environmental_score_decline_rate'].apply(lambda x: np.where(x > 0.5, 1, 0)[0]).to_csv('./factset_revere_companies/Environmental_Dimension/3-c-firm.csv')

In [55]:
first_degree_supplier.to_csv('./factset_revere_companies/Environmental_Dimension/year/1-s.csv')
second_degree_supplier.to_csv('./factset_revere_companies/Environmental_Dimension/year/2-s.csv')
third_degree_supplier.to_csv('./factset_revere_companies/Environmental_Dimension/year/3-s.csv')
first_degree_customer.to_csv('./factset_revere_companies/Environmental_Dimension/year/1-c.csv')
second_degree_customer.to_csv('./factset_revere_companies/Environmental_Dimension/year/2-c.csv')
third_degree_customer.to_csv('./factset_revere_companies/Environmental_Dimension/year/3-c.csv')